## Preprocesamiento de noticias con Pyspark

### Importar Librerias 

In [1]:
!pip install emoji tweet-preprocessor tqdm -q
!python -m nltk.downloader punkt -q
!python -m nltk.downloader averaged_perceptron_tagger -q
!python -m nltk.downloader stopwords -q


/opt/conda/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
/opt/conda/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
/opt/conda/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))


In [9]:
import pandas as pd
import pyspark
import json
import preprocessor as tweet_preprocesor
tweet_preprocesor.set_options(
    tweet_preprocesor.OPT.ESCAPE_CHAR,
    tweet_preprocesor.OPT.URL,
    tweet_preprocesor.OPT.URL)

import nltk
STOPWORDS = nltk.corpus.stopwords.words('english')

import emoji
import re


from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,size, length,col
from pyspark.sql.types import StringType,DoubleType

In [3]:
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "20g") \
    .appName('my-cool-app') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/21 00:49:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Cargar la base de datos

In [4]:
with open('news_new_format_2021-08-18.json','r',encoding='utf8') as f:
    news = f.readlines()

news = [json.loads(a.replace('\n','')) for a in news]
news[0]

{'_id': {'$oid': '611d9fe1049812a18b13a8dd'},
 'topic': 'cultura',
 'text': '',
 'title': "El vallenato con guitarra de los hijos de 'La celosa'",
 'year': 2020,
 'month': 12,
 'day': 7,
 'key': 'cultura-musica-y-libros-vallenatos-de-el-trio-de-oro-32677',
 'url': 'http://www.eltiempo.com//cultura/musica-y-libros/vallenatos-de-el-trio-de-oro-32677'}

### Limpieza inicial de los datos 

In [5]:
features = list(news[0].keys())
data = [[value for key,value in doc.items()] for doc in news] 

In [6]:
df = spark.createDataFrame(data, features)
df=df.filter(length(col('text'))>0)
df=df.dropDuplicates(['text'])

### Prepocesamiento de las noticias 

In [14]:
def clean_text(text,tokenize=False,demojize=True,only_nouns=False):
    data = tweet_preprocesor.clean(text)
    data = re.sub(r'[,!?;-]+', '.', text)
    data = nltk.word_tokenize(data)  # tokenize string to words
    data = [ ch.lower() for ch in data
             if ch.isalpha()
             or ch == '.'
             or emoji.get_emoji_regexp().search(ch)
           ]
    if demojize:
      data = [ emoji.demojize(ch) 
               if emoji.get_emoji_regexp().search(ch)
               else ch
               for ch in data
      ]
    if only_nouns:
      is_noun = lambda pos: pos[:2] in ('NN')
      data = [
              word for (word, post) in 
              nltk.pos_tag(data) if 
              is_noun(post) and
              word not in STOPWORDS
      ]
    if not tokenize:
      data = " ".join(data)
    return data

In [15]:
clean_text_udf=udf(clean_text,StringType())

In [16]:
df=df.withColumn('text_clean',clean_text_udf(df['text']))

In [17]:
df.select('text','text_clean').show(3)

21/11/21 01:00:46 WARN TaskSetManager: Stage 6 contains a task of very large size (161144 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+
|                text|          text_clean|
+--------------------+--------------------+
|A Mintic, el papá...|a mintic . el pap...|
|Adriana Lucía es ...|adriana lucía es ...|
|Alrededor del mun...|alrededor del mun...|
+--------------------+--------------------+
only showing top 3 rows

